In [ ]:
!pip3 install python-terrier ir-datasets git+https://github.com/tira-io/tira.git@development#\&subdirectory=python-client

  Cloning https://github.com/tira-io/tira.git (to revision development) to /tmp/pip-req-build-p2wvdwgz
  Running command git clone --filter=blob:none --quiet https://github.com/tira-io/tira.git /tmp/pip-req-build-p2wvdwgz
  Running command git checkout -b development --track origin/development
  Switched to a new branch 'development'
  Branch 'development' set up to track remote branch 'development' from 'origin'.
  Resolved https://github.com/tira-io/tira.git to commit e06d51d38ac6b5c95d3bf5735631f5ca34fb1586
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.2/335.2 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded
ensure_pyterrier_is_loaded()
import pyterrier as pt
from tqdm import tqdm
import pandas as pd

from tira.rest_api_client import Client
tira = Client()

terrier-assemblies 5.7 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.7 jar not found, downloading to /root/.pyterrier...
Done
terrier-prf -SNAPSHOT jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.0 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



In [ ]:
dataset = pt.get_dataset("irds:ir-benchmarks/antique-test-20230107-training")
topics = dataset.get_topics('query')
topics[topics['qid'] == '1783010']

,qid,query
9,1783010,what is blaphsemy


In [ ]:
index = tira.get_run_output('ir-benchmarks/tira-ir-starter/Index (tira-ir-starter-pyterrier)', 'antique-test-20230107-training') + '/index'
index = pt.IndexFactory.of(index)

bm25 = pt.BatchRetrieve(index, wmodel="BM25")
pl2 = pt.BatchRetrieve(index, wmodel="PL2")

In [ ]:
dataset = pt.get_dataset("irds:ir-benchmarks/msmarco-passage-trec-dl-2019-judged-20230107-training")
index = tira.get_run_output('ir-benchmarks/tira-ir-starter/Index (tira-ir-starter-pyterrier)', 'msmarco-passage-trec-dl-2019-judged-20230107-training') + '/index'
index = pt.IndexFactory.of(index)

bm25 = pt.BatchRetrieve(index, wmodel="BM25")
pl2 = pt.BatchRetrieve(index, wmodel="PL2")

Download from the Incubator: https://files.webis.de/data-in-production/data-research/tira-zenodo-dump-preparation/pyterrier-indexes/2023-01-07-22-09-56.zip
	This is only used for last spot checks before archival to Zenodo.


Download: 100%|██████████| 892M/892M [00:01<00:00, 581MiB/s]


Download finished. Extract...
Extraction finished:  /root/.tira/extracted_runs/ir-benchmarks/msmarco-passage-trec-dl-2019-judged-20230107-training/tira-ir-starter


In [ ]:
tokeniser = pt.autoclass("org.terrier.indexing.tokenisation.Tokeniser").getTokeniser()
def pt_tokenise(text):
    return ' '.join(tokeniser.getTokens(text))


In [ ]:
hunspell = tira.pt.transform_queries('workshop-on-open-web-search/qspell/hunspell', 'msmarco-passage-trec-dl-2019-judged-20230107-training')
pyspell = tira.pt.transform_queries('workshop-on-open-web-search/qspell/pyspell', 'msmarco-passage-trec-dl-2019-judged-20230107-training')
rushton = tira.pt.transform_queries('workshop-on-open-web-search/qspell/rushton', 'msmarco-passage-trec-dl-2019-judged-20230107-training')
chat_gpt = tira.pt.transform_queries('ir-benchmarks/qspell/ChatGPT', 'msmarco-passage-trec-dl-2019-judged-20230107-training')
toks = pt.apply.query(lambda i: pt_tokenise(i['query']))

Download: 3.67kiB [00:00, 25.3MiB/s]


Download finished. Extract...
Extraction finished:  /root/.tira/extracted_runs/workshop-on-open-web-search/msmarco-passage-trec-dl-2019-judged-20230107-training/qspell


Download: 3.80kiB [00:00, 25.1MiB/s]


Download finished. Extract...
Extraction finished:  /root/.tira/extracted_runs/workshop-on-open-web-search/msmarco-passage-trec-dl-2019-judged-20230107-training/qspell


Download: 3.72kiB [00:00, 27.5MiB/s]


Download finished. Extract...
Extraction finished:  /root/.tira/extracted_runs/workshop-on-open-web-search/msmarco-passage-trec-dl-2019-judged-20230107-training/qspell


Download: 3.60kiB [00:00, 25.1MiB/s]

Download finished. Extract...
Extraction finished:  /root/.tira/extracted_runs/ir-benchmarks/msmarco-passage-trec-dl-2019-judged-20230107-training/qspell


In [ ]:
systems = [
    bm25,
    pl2,
    hunspell >> toks >> bm25,
    hunspell >> toks >> pl2,
    pyspell >> toks >> bm25,
    pyspell >> toks >> pl2,
    rushton >> toks >> bm25,
    rushton >> toks >> pl2,
    chat_gpt >> toks >> bm25,
    chat_gpt >> toks >> pl2,
]
names = ['BM25', 'PL2', 'hunspell >> BM25', 'hunspell >> PL2', 'pyspell >> BM25', 'pyspell >> PL2', 'rushton >> BM25', 'rushton >> PL2', 'ChatGPT >> BM25', 'ChatGPT >> PL2']

pt.Experiment(systems, dataset.get_topics('query'), dataset.get_qrels(), ['ndcg_cut.10'], names=names, verbose=True)

pt.Experiment:   0%|          | 0/10 [00:00<?, ?system/s]

,name,ndcg_cut.10
0,BM25,0.479540
1,PL2,0.466875
2,hunspell >> BM25,0.356071
3,hunspell >> PL2,0.337754
4,pyspell >> BM25,0.312993
5,pyspell >> PL2,0.302877
6,rushton >> BM25,0.482312
7,rushton >> PL2,0.467155
8,ChatGPT >> BM25,0.478834
9,ChatGPT >> PL2,0.465025


In [ ]:
hunspell = tira.pt.transform_queries('workshop-on-open-web-search/qspell/hunspell', 'msmarco-passage-trec-dl-2020-judged-20230107-training')
pyspell = tira.pt.transform_queries('workshop-on-open-web-search/qspell/pyspell', 'msmarco-passage-trec-dl-2020-judged-20230107-training')
rushton = tira.pt.transform_queries('workshop-on-open-web-search/qspell/rushton', 'msmarco-passage-trec-dl-2020-judged-20230107-training')
chat_gpt = tira.pt.transform_queries('ir-benchmarks/qspell/ChatGPT', 'msmarco-passage-trec-dl-2020-judged-20230107-training')
toks = pt.apply.query(lambda i: pt_tokenise(i['query']))

Download: 3.97kiB [00:00, 3.01MiB/s]


Download finished. Extract...
Extraction finished:  /root/.tira/extracted_runs/workshop-on-open-web-search/msmarco-passage-trec-dl-2020-judged-20230107-training/qspell


Download: 4.11kiB [00:00, 3.22MiB/s]


Download finished. Extract...
Extraction finished:  /root/.tira/extracted_runs/workshop-on-open-web-search/msmarco-passage-trec-dl-2020-judged-20230107-training/qspell


Download: 4.00kiB [00:00, 29.6MiB/s]


Download finished. Extract...
Extraction finished:  /root/.tira/extracted_runs/workshop-on-open-web-search/msmarco-passage-trec-dl-2020-judged-20230107-training/qspell


Download: 3.90kiB [00:00, 8.72MiB/s]

Download finished. Extract...
Extraction finished:  /root/.tira/extracted_runs/ir-benchmarks/msmarco-passage-trec-dl-2020-judged-20230107-training/qspell


In [ ]:
dataset = pt.get_dataset("irds:ir-benchmarks/msmarco-passage-trec-dl-2020-judged-20230107-training")
systems = [
    bm25,
    pl2,
    hunspell >> toks >> bm25,
    hunspell >> toks >> pl2,
    pyspell >> toks >> bm25,
    pyspell >> toks >> pl2,
    rushton >> toks >> bm25,
    rushton >> toks >> pl2,
    chat_gpt >> toks >> bm25,
    chat_gpt >> toks >> pl2,
]
names = ['BM25', 'PL2', 'hunspell >> BM25', 'hunspell >> PL2', 'pyspell >> BM25', 'pyspell >> PL2', 'rushton >> BM25', 'rushton >> PL2', 'ChatGPT >> BM25', 'ChatGPT >> PL2']

pt.Experiment(systems, dataset.get_topics('query'), dataset.get_qrels(), ['ndcg_cut.10'], names=names, verbose=True)

Download: 235kiB [00:00, 111MiB/s]


Download finished. Extract...
Extraction finished:  /root/.tira/extracted_datasets/ir-benchmarks/msmarco-passage-trec-dl-2020-judged-20230107-training/


pt.Experiment:   0%|          | 0/10 [00:00<?, ?system/s]

,name,ndcg_cut.10
0,BM25,0.493627
1,PL2,0.483847
2,hunspell >> BM25,0.417859
3,hunspell >> PL2,0.408410
4,pyspell >> BM25,0.333832
5,pyspell >> PL2,0.324871
6,rushton >> BM25,0.493627
7,rushton >> PL2,0.483847
8,ChatGPT >> BM25,0.482022
9,ChatGPT >> PL2,0.470418


In [ ]:
systems = [
    bm25,
    pl2,
    hunspell >> toks >> bm25,
    hunspell >> toks >> pl2,
    pyspell >> toks >> bm25,
    pyspell >> toks >> pl2,
    rushton >> toks >> bm25,
    rushton >> toks >> pl2,
    chat_gpt >> toks >> bm25,
    chat_gpt >> toks >> pl2,
]
names = ['BM25', 'PL2', 'hunspell >> BM25', 'hunspell >> PL2', 'pyspell >> BM25', 'pyspell >> PL2', 'rushton >> BM25', 'rushton >> PL2', 'ChatGPT >> BM25', 'ChatGPT >> PL2']

pt.Experiment(systems, dataset.get_topics('query'), dataset.get_qrels(), ['ndcg_cut.10'], names=names, verbose=True)

pt.Experiment:   0%|          | 0/10 [00:00<?, ?system/s]

,name,ndcg_cut.10
0,BM25,0.510402
1,PL2,0.488955
2,hunspell >> BM25,0.480913
3,hunspell >> PL2,0.458890
4,pyspell >> BM25,0.433223
5,pyspell >> PL2,0.410252
6,rushton >> BM25,0.509530
7,rushton >> PL2,0.488457
8,ChatGPT >> BM25,0.490155
9,ChatGPT >> PL2,0.470942


# Oracle

In [ ]:
dataset = pt.get_dataset("irds:ir-benchmarks/msmarco-passage-trec-dl-2020-judged-20230107-training")
systems = [
    pl2,
    hunspell >> toks >> pl2,
    pyspell >> toks >> pl2,
    rushton >> toks >> pl2,
    chat_gpt >> toks >> pl2,
]
names = ['PL2', 'hunspell >> PL2', 'pyspell >> PL2', 'rushton >> PL2', 'ChatGPT >> PL2']
df_eval = pt.Experiment(systems, dataset.get_topics('query'), dataset.get_qrels(), ['ndcg_cut.10'], names=names, verbose=True, perquery=True)
df_eval

Download: 235kiB [00:00, 168MiB/s]

Download finished. Extract...
Extraction finished:  /root/.tira/extracted_datasets/ir-benchmarks/msmarco-passage-trec-dl-2020-judged-20230107-training/


pt.Experiment:   0%|          | 0/5 [00:00<?, ?system/s]

,name,qid,measure,value
216,ChatGPT >> PL2,1030303,ndcg_cut.10,0.872746
217,ChatGPT >> PL2,1037496,ndcg_cut.10,0.520479
218,ChatGPT >> PL2,1043135,ndcg_cut.10,0.601138
219,ChatGPT >> PL2,1051399,ndcg_cut.10,0.164867
220,ChatGPT >> PL2,1064670,ndcg_cut.10,0.773036
...,...,...,...,...
211,rushton >> PL2,911232,ndcg_cut.10,0.670264
212,rushton >> PL2,914916,ndcg_cut.10,0.512427
213,rushton >> PL2,938400,ndcg_cut.10,0.128349
214,rushton >> PL2,940547,ndcg_cut.10,0.165437


In [ ]:
# The oracle (that always chooses the best variant out of "no spelling correction", "hunspell", "rusthon", "chatGPT")
df_eval.groupby('qid').agg({'value': 'max'}).reset_index()['value'].mean()

0.49665603388228124

In [ ]:
dataset = pt.get_dataset("irds:ir-benchmarks/msmarco-passage-trec-dl-2019-judged-20230107-training")
systems = [
    pl2,
    hunspell >> toks >> pl2,
    pyspell >> toks >> pl2,
    rushton >> toks >> pl2,
    chat_gpt >> toks >> pl2,
]
names = ['PL2', 'hunspell >> PL2', 'pyspell >> PL2', 'rushton >> PL2', 'ChatGPT >> PL2']
df_eval = pt.Experiment(systems, dataset.get_topics('query'), dataset.get_qrels(), ['ndcg_cut.10'], names=names, verbose=True, perquery=True)
df_eval

Download: 199kiB [00:00, 162MiB/s]


Download finished. Extract...
Extraction finished:  /root/.tira/extracted_datasets/ir-benchmarks/msmarco-passage-trec-dl-2019-judged-20230107-training/


pt.Experiment:   0%|          | 0/5 [00:00<?, ?system/s]

,name,qid,measure,value
181,ChatGPT >> PL2,1037798,ndcg_cut.10,0.131672
193,ChatGPT >> PL2,104861,ndcg_cut.10,0.745783
174,ChatGPT >> PL2,1063750,ndcg_cut.10,0.112485
201,ChatGPT >> PL2,1103812,ndcg_cut.10,0.501800
198,ChatGPT >> PL2,1106007,ndcg_cut.10,0.375959
...,...,...,...,...
163,rushton >> PL2,855410,ndcg_cut.10,0.881199
159,rushton >> PL2,87181,ndcg_cut.10,0.583667
162,rushton >> PL2,87452,ndcg_cut.10,0.456956
139,rushton >> PL2,915593,ndcg_cut.10,0.475575


In [ ]:
# The oracle (that always chooses the best variant out of "no spelling correction", "hunspell", "rusthon", "chatGPT")
df_eval.groupby('qid').agg({'value': 'max'}).reset_index()['value'].mean()

0.4671554027880264

In [ ]:
systems = [
    bm25,
    hunspell >> toks >> bm25,
    pyspell >> toks >> bm25,
    rushton >> toks >> bm25,
    chat_gpt >> toks >> bm25,
]
names = ['BM25', 'hunspell >> BM25', 'pyspell >> BM25', 'rushton >> BM25', 'ChatGPT >> BM25']
df_eval = pt.Experiment(systems, dataset.get_topics('query'), dataset.get_qrels(), ['ndcg_cut.10'], names=names, verbose=True, perquery=True)
df_eval

pt.Experiment:   0%|          | 0/5 [00:00<?, ?system/s]

,name,qid,measure,value
0,BM25,1030303,ndcg_cut.10,0.872746
1,BM25,1037496,ndcg_cut.10,0.542564
2,BM25,1043135,ndcg_cut.10,0.595064
3,BM25,1051399,ndcg_cut.10,0.179623
4,BM25,1064670,ndcg_cut.10,0.771352
...,...,...,...,...
211,rushton >> BM25,911232,ndcg_cut.10,0.670264
212,rushton >> BM25,914916,ndcg_cut.10,0.512427
213,rushton >> BM25,938400,ndcg_cut.10,0.114339
214,rushton >> BM25,940547,ndcg_cut.10,0.127349


In [ ]:
# The oracle (that always chooses the best variant out of "no spelling correction", "hunspell", "rusthon", "chatGPT")
df_eval.groupby('qid').agg({'value': 'max'}).reset_index()['value'].mean()

0.5068656659357653

In [ ]:
systems = [
    bm25,
    hunspell >> toks >> bm25,
    pyspell >> toks >> bm25,
    rushton >> toks >> bm25,
    chat_gpt >> toks >> bm25,
]
names = ['BM25', 'hunspell >> BM25', 'pyspell >> BM25', 'rushton >> BM25', 'ChatGPT >> BM25']
df_eval = pt.Experiment(systems, dataset.get_topics('query'), dataset.get_qrels(), ['ndcg_cut.10'], names=names, verbose=True, perquery=True)
df_eval

pt.Experiment:   0%|          | 0/5 [00:00<?, ?system/s]

,name,qid,measure,value
9,BM25,1037798,ndcg_cut.10,0.152866
21,BM25,104861,ndcg_cut.10,0.776306
2,BM25,1063750,ndcg_cut.10,0.142811
29,BM25,1103812,ndcg_cut.10,0.507699
26,BM25,1106007,ndcg_cut.10,0.368773
...,...,...,...,...
163,rushton >> BM25,855410,ndcg_cut.10,0.881199
159,rushton >> BM25,87181,ndcg_cut.10,0.579026
162,rushton >> BM25,87452,ndcg_cut.10,0.498614
139,rushton >> BM25,915593,ndcg_cut.10,0.417021


In [ ]:
# The oracle (that always chooses the best variant out of "no spelling correction", "hunspell", "rusthon", "chatGPT")
df_eval.groupby('qid').agg({'value': 'max'}).reset_index()['value'].mean()

0.4828546051406672

In [ ]:
systems = [
    bm25,
    hunspell >> toks >> bm25,
    pyspell >> toks >> bm25,
    rushton >> toks >> bm25,
    chat_gpt >> toks >> bm25,
]
names = ['BM25', 'hunspell >> BM25', 'pyspell >> BM25', 'rushton >> BM25', 'ChatGPT >> BM25']
df_eval = pt.Experiment(systems, dataset.get_topics('query'), dataset.get_qrels(), ['ndcg_cut.10'], names=names, verbose=True, perquery=True)
df_eval

pt.Experiment:   0%|          | 0/5 [00:00<?, ?system/s]

,name,qid,measure,value
70,BM25,100653,ndcg_cut.10,0.396720
85,BM25,1015624,ndcg_cut.10,0.575231
43,BM25,1017690,ndcg_cut.10,0.754357
116,BM25,1035857,ndcg_cut.10,0.654812
122,BM25,103830,ndcg_cut.10,0.513426
...,...,...,...,...
619,rushton >> BM25,851124,ndcg_cut.10,0.577386
695,rushton >> BM25,896725,ndcg_cut.10,0.380379
704,rushton >> BM25,922849,ndcg_cut.10,0.565539
778,rushton >> BM25,949154,ndcg_cut.10,0.664721


In [ ]:
# The oracle (that always chooses the best variant out of "no spelling correction", "hunspell", "rusthon", "chatGPT")
df_eval.groupby('qid').agg({'value': 'max'}).reset_index()['value'].mean()

0.524358154006209

In [ ]:
systems = [
    pl2,
    hunspell >> toks >> pl2,
    pyspell >> toks >> pl2,
    rushton >> toks >> pl2,
    chat_gpt >> toks >> pl2,
]
names = ['PL2', 'hunspell >> PL2', 'pyspell >> PL2', 'rushton >> PL2', 'ChatGPT >> PL2']
df_eval = pt.Experiment(systems, dataset.get_topics('query'), dataset.get_qrels(), ['ndcg_cut.10'], names=names, verbose=True, perquery=True)
df_eval

pt.Experiment:   0%|          | 0/5 [00:00<?, ?system/s]

,name,qid,measure,value
870,ChatGPT >> PL2,100653,ndcg_cut.10,0.396720
885,ChatGPT >> PL2,1015624,ndcg_cut.10,0.638852
843,ChatGPT >> PL2,1017690,ndcg_cut.10,0.750770
916,ChatGPT >> PL2,1035857,ndcg_cut.10,0.645368
922,ChatGPT >> PL2,103830,ndcg_cut.10,0.563644
...,...,...,...,...
619,rushton >> PL2,851124,ndcg_cut.10,0.537939
695,rushton >> PL2,896725,ndcg_cut.10,0.362022
704,rushton >> PL2,922849,ndcg_cut.10,0.505162
778,rushton >> PL2,949154,ndcg_cut.10,0.664721


In [ ]:
# The oracle (that always chooses the best variant out of "no spelling correction", "hunspell", "rusthon", "chatGPT")
df_eval.groupby('qid').agg({'value': 'max'}).reset_index()['value'].mean()

0.5028046380905374

# Statistical Analysis

In [ ]:
def count_spelling_differences(dataset):
  topics = pt.get_dataset(f"irds:ir-benchmarks/{dataset}").get_topics('query')
  topics = {i['qid']: i['query'] for _, i in topics.iterrows()}

  hunspell = tira.pt.transform_queries('workshop-on-open-web-search/qspell/hunspell', dataset)
  topics_corrected = hunspell(pt.get_dataset(f"irds:ir-benchmarks/{dataset}").get_topics('query'))
  topics_corrected = {i['qid']: i['query'] for _, i in topics_corrected.iterrows()}

  pyspell = tira.pt.transform_queries('workshop-on-open-web-search/qspell/pyspell', dataset)
  topics_corrected_pyspell = pyspell(pt.get_dataset(f"irds:ir-benchmarks/{dataset}").get_topics('query'))
  topics_corrected_pyspell = {i['qid']: i['query'] for _, i in topics_corrected_pyspell.iterrows()}

  rushton = tira.pt.transform_queries('workshop-on-open-web-search/qspell/rushton', dataset)
  topics_corrected_rushton = rushton(pt.get_dataset(f"irds:ir-benchmarks/{dataset}").get_topics('query'))
  topics_corrected_rushton = {i['qid']: i['query'] for _, i in topics_corrected_rushton.iterrows()}

  spelling_differences_tokenized_hunspell = 0
  spelling_differences_tokenized_pyspell = 0
  spelling_differences_tokenized_rushton = 0

  for qid in topics:
    orignal_query = topics[qid]

    spell_corrected_query = topics_corrected[qid]
    spelling_differences_tokenized_hunspell += 0 if pt_tokenise(orignal_query).lower() == pt_tokenise(spell_corrected_query).lower() else 1

    spell_corrected_query = topics_corrected_pyspell[qid]
    spelling_differences_tokenized_pyspell += 0 if pt_tokenise(orignal_query).lower() == pt_tokenise(spell_corrected_query).lower() else 1

    spell_corrected_query = topics_corrected_rushton[qid]
    spelling_differences_tokenized_rushton += 0 if pt_tokenise(orignal_query).lower() == pt_tokenise(spell_corrected_query).lower() else 1


  return {
      'num_queries': len(topics),
      'spelling_differences_hunspell': spelling_differences_tokenized_hunspell,
      'spelling_differences_pyspell': spelling_differences_tokenized_pyspell,
      'spelling_differences_rusthon': spelling_differences_tokenized_rushton,
  }


In [ ]:
datasets = [
    'antique-test-20230107-training', 'argsme-touche-2021-task-1-20230209-training', 'argsme-touche-2020-task-1-20230209-training',
    'clueweb09-en-trec-web-2009-20230107-training', 'clueweb09-en-trec-web-2010-20230107-training', 'clueweb09-en-trec-web-2011-20230107-training',
    'clueweb09-en-trec-web-2012-20230107-training', 'clueweb12-touche-2020-task-2-20230209-training', 'clueweb12-touche-2021-task-2-20230209-training',
    'clueweb12-trec-misinfo-2019-20240214-training', 'clueweb12-trec-web-2013-20230107-training', 'clueweb12-trec-web-2014-20230107-training',
    'cord19-fulltext-trec-covid-20230107-training', 'cranfield-20230107-training', 'disks45-nocr-trec-robust-2004-20230209-training',
    'disks45-nocr-trec7-20230209-training', 'disks45-nocr-trec8-20230209-training', 'gov-trec-web-2002-20230209-training',
    'gov-trec-web-2003-20230209-training', 'gov-trec-web-2004-20230209-training', 'gov2-trec-tb-2004-20230209-training',
    'gov2-trec-tb-2005-20230209-training', 'gov2-trec-tb-2006-20230209-training', 'longeval-heldout-20230513-training',
    'longeval-long-september-20230513-training', 'longeval-short-july-20230513-training', 'longeval-train-20230513-training',
    'medline-2004-trec-genomics-2004-20230107-training', 'medline-2004-trec-genomics-2005-20230107-training', 'medline-2017-trec-pm-2017-20230211-training',
    'medline-2017-trec-pm-2018-20230211-training', 'msmarco-passage-trec-dl-2019-judged-20230107-training', 'msmarco-passage-trec-dl-2020-judged-20230107-training',
    'nfcorpus-test-20230107-training', 'trec-tip-of-the-tongue-dev-20230607-training', 'vaswani-20230107-training',
    'wapo-v2-trec-core-2018-20230107-training'
]

df = []
for dataset in tqdm(datasets):
  i = count_spelling_differences(dataset)
  i['dataset']= dataset
  df += [i]

df = pd.DataFrame(df)

NameError: name 'tqdm' is not defined

In [ ]:
print(df[['num_queries', 'spelling_differences_rusthon']].sum())

print(107/5320)

num_queries                     5320
spelling_differences_rusthon     107
dtype: int64
0.020112781954887218


In [ ]:
df['Errors in Percent (Hunspell)'] = df['spelling_differences_hunspell'] / df['num_queries']
df['Errors in Percent (PySpell)'] = df['spelling_differences_pyspell'] / df['num_queries']
df['Errors in Percent (Rushton)'] = df['spelling_differences_rusthon'] / df['num_queries']
df[['dataset', 'num_queries', 'Errors in Percent (Rushton)', 'spelling_differences_rusthon',  'Errors in Percent (Hunspell)', 'spelling_differences_hunspell', 'Errors in Percent (PySpell)', 'spelling_differences_pyspell']].sort_values('Errors in Percent (Rushton)')


,dataset,num_queries,Errors in Percent (Rushton),spelling_differences_rusthon,Errors in Percent (Hunspell),spelling_differences_hunspell,Errors in Percent (PySpell),spelling_differences_pyspell
18,gov-trec-web-2003-20230209-training,50,0.000000,0,0.060000,3,0.100000,5
33,nfcorpus-test-20230107-training,325,0.000000,0,0.332308,108,0.298462,97
32,msmarco-passage-trec-dl-2020-judged-20230107-t...,54,0.000000,0,0.203704,11,0.425926,23
27,medline-2004-trec-genomics-2004-20230107-training,50,0.000000,0,0.540000,27,0.720000,36
23,longeval-heldout-20230513-training,98,0.000000,0,0.244898,24,0.367347,36
22,gov2-trec-tb-2006-20230209-training,50,0.000000,0,0.060000,3,0.360000,18
21,gov2-trec-tb-2005-20230209-training,50,0.000000,0,0.100000,5,0.280000,14
20,gov2-trec-tb-2004-20230209-training,50,0.000000,0,0.120000,6,0.240000,12
19,gov-trec-web-2004-20230209-training,225,0.000000,0,0.151111,34,0.355556,80
35,vaswani-20230107-training,93,0.000000,0,0.064516,6,0.096774,9


In [ ]:
count_spelling_differences()

There are multiple query fields available: ('text', 'title', 'query', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.
There are multiple query fields available: ('text', 'title', 'query', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.


{'num_queries': 49,
 'spelling_differences': 49,
 'spelling_differences_after_tokenization': 0}

In [ ]:
df = dataset.get_topics()
df['orig_query'] = df['query'].apply(lambda i: ' '.join(tokeniser.getTokens(i)))
df = hunspell(df)
del df['query_0']
df[df['orig_query'] != df['query']]

,qid,orig_query,query
0,3990512,how can we get concentration onsomething,how can we get concentration something
2,2528767,how do i determine the charge of the iron ion ...,how do i determine the charge of the iron ion ...
4,1880028,what does see leaflet mean on ept pregnancy test,what does see leaflet mean on ep pregnancy test
6,1287437,how can i lose 30 pounds by june3,how can i lose 30 pounds by jejune
7,2956570,what are the words to write the sound of raind...,what are the words to write the sound of raind...
...,...,...,...
189,1509982,how is th chemistry is a basic of science,how is ht chemistry is a basic of science
194,1850323,why some men never wash their hands after usin...,why some men never wash their hands after usin...
197,1582877,why do we go to school if in the real world we...,why do we go to school if in the real world we...
198,1340574,why do some people only go to church on easter...,why do some people only go to church on easter...


,qid,orig_query,query
0,3990512,how can we get concentration onsomething,how can we get concentration something
2,2528767,how do i determine the charge of the iron ion ...,how do i determine the charge of the iron ion ...
4,1880028,what does see leaflet mean on ept pregnancy test,what does see leaflet mean on ep pregnancy test
6,1287437,how can i lose 30 pounds by june3,how can i lose 30 pounds by jejune
7,2956570,what are the words to write the sound of raind...,what are the words to write the sound of raind...
...,...,...,...
189,1509982,how is th chemistry is a basic of science,how is ht chemistry is a basic of science
194,1850323,why some men never wash their hands after usin...,why some men never wash their hands after usin...
197,1582877,why do we go to school if in the real world we...,why do we go to school if in the real world we...
198,1340574,why do some people only go to church on easter...,why do some people only go to church on easter...


In [ ]:
(dataset.get_topics()['query'] == hunspell(dataset.get_topics())['query']).value_counts()

True     136
False     64
Name: query, dtype: int64

In [ ]:
dataset.get_topics()['query'].iloc[1]

'why doesn t the water fall off earth if it s round'

'why doesn t the water fall off earth if it s round'